In [1]:
from CoolProp.CoolProp import PropsSI as COOLPROP
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import get_test_data

#Listas
registro_fluidos = []
COP_lista = []
lista_T5 = []
Wcomp_total_lista = []
cont = []

#Lista de Temperaturas
T5 = 256.15

#Lista de características para as curvas do gráfico
lista_cor = ['r','g','b', 'c', 'm']
lista_ls = ['-', '-.', ':']
caracteristicas = []
for ls in lista_ls:
    for cor in lista_cor:
        caracteristicas.append((cor, ls))   
    
#Dados do problema
Pfrig = 120 * 1000
T1 = -80 + 273.15
T11 = 28 + 273.15
N = 0.7 #Eficiência isentrópica
fluid_high_lista = ['R134a', 'R717', 'R290', 'R600']
fluid_low_lista = ['R23', 'R170', 'NitrousOxide']
drop_suc = 0 #[Pa]
drop_desc = 0 #[Pa]
DeltaT = 0.01 #[°C]

#Loop nos dois ciclos - variação dos refrigerantes e do coeficiente politrópico
for fluid_low in fluid_low_lista:
    for fluid_high in fluid_high_lista:
        par_fluido = (fluid_low, fluid_high)
        registro_fluidos.append(par_fluido)
        #Pressão crítica do fluido superior
        P_crit_high = COOLPROP('PCRIT', fluid_high)
        #Ciclo inferior
        P1 = COOLPROP('P', 'T', T1, 'Q', 1, fluid_low)
        P5 = COOLPROP('P', 'T', T5, 'Q', 0, fluid_low)
        h5 = COOLPROP('H', 'T', T5, 'Q', 0, fluid_low)
        h6 = h5
        #Pressão intermediária
        P_int =  (P1 * P5) ** 0.5
        P6 = P_int
        P3 = P_int
        P7 = P_int
        P2 = P_int
        h7 = COOLPROP('H', 'P', P7, 'Q', 0, fluid_low)
        h8 = h7
        h3 = COOLPROP('H', 'P', P3, 'Q', 1, fluid_low)
        h1 = COOLPROP('H', 'T', T1, 'Q', 1, fluid_low)
        #CBP
        ha = h1
        Pa = P1 - drop_suc
        Ta = COOLPROP('T', 'H', ha, 'P', Pa, fluid_low)
        Pb = Pa
        Tb = Ta + DeltaT
        hb = COOLPROP('H', 'T', Tb, 'P', Pb, fluid_low)
        Sb = COOLPROP('S', 'T', Tb, 'P', Pb, fluid_low)
        Pc = P2 + drop_desc
        Sc_ideal = Sb     
        hc_ideal = COOLPROP('H', 'S', Sc_ideal , 'P', Pc, fluid_low)
        hc_real = (hc_ideal - hb)/N + hb
        h2 = hc_real
            
        m_CBP = Pfrig / (h1 - h8)
        #CAP
        hw = h3
        Pw = P3 - drop_suc
        Tw = COOLPROP('T', 'H', hw, 'P', Pw, fluid_low)
        Px = Pw
        Tx = Tw + DeltaT
        hx = COOLPROP('H', 'P', Px, 'T', Tx, fluid_low)
        sx = COOLPROP('S', 'P', Px, 'T', Tx, fluid_low)
        P4 = P5
        Py = P4 + drop_desc
        sy_ideal = sx
        hy_ideal = COOLPROP('H', 'P', Py, 'S', sy_ideal, fluid_low)
        hy_real = (hy_ideal - hx)/N + hx
        h4 = hy_real
        
        #Ciclo superior
        h11 = COOLPROP('H', 'T', T11, 'Q', 0, fluid_high)
        T12 = T5 - 5
        h12 = h11 
        P12 = COOLPROP('P', 'T', T12, 'Q', 0, fluid_high)
        
        P9 = P12
        h9 = COOLPROP('H', 'P', P9, 'Q', 1, fluid_high)

        he = h9
        Pe = P9 - drop_suc
        Te = COOLPROP ('T', 'H', he, 'P', Pe, fluid_high)
        Pf = Pe
        Tf = Te + DeltaT
        hf = COOLPROP('H', 'P', Pf, 'T', Tf, fluid_high)
        sf = COOLPROP('S', 'P', Pf, 'T', Tf, fluid_high)
    
        P11 = COOLPROP('P', 'T', T11, 'Q', 0, fluid_high)
        P10 = P11
        Pg = P10 + drop_desc
        sg_ideal = sf
        hg_ideal = COOLPROP('H', 'P', Pg, 'S', sg_ideal, fluid_high)
        hg_real = (hg_ideal - hf)/N + hf
        

        #Trabalho específico do CBP
        w_CBP = hc_real - hb
        #Trabalho do CBP
        W_CBP = m_CBP * w_CBP
        #CAP
        m_CAP = m_CBP * (h7 - h2) / (h6 - h3)
        #Trabalho específico do CAP
        w_CAP = hy_real - hx
        #Trabalho do CAP
        W_CAP = m_CAP * w_CAP
        #Trabalho total no ciclo inferior
        W_low_total = W_CAP + W_CBP
        #Calor trocado no trocador central
        Qcond_low =  m_CAP * (h4 - h5)
        #Ciclo superior
        Qevap_high = Qcond_low
        m_flow_high = Qevap_high / (h9 - h12)
        #Verificação da pressão crítica
        if P_crit_high > P10:
            lista_T5.append(T5)
            #Trabalho específico do compressor do ciclo superior
            w_high = hg_real - hf
            #Trabalho do compressor do ciclo superior
            Wcomp_high = m_flow_high * w_high
            #Trabalho total
            Wcomp_total = Wcomp_high + W_low_total
            Wcomp_total_lista.append(Wcomp_total / 1000)
            #COP
            COP = Pfrig / Wcomp_total
            COP_lista.append(COP)
            print(COP)
            print(P_int)
            print("A", fluid_high)
            print("B", fluid_low)
            print("P4",P4)
        else:
            print ('A pressão P10 = {:0.2f} kPa (T5 = {}, fluido inf = {}, fluido sup = {}) é maior do que a ' 
           'pressão crítica do {} e, por isso, o ponto não foi incluído no gráfico'.format(P10 / 1000,T5,fluid_low,
                                                                                               fluid_high, fluid_high))               



0.8604643057660354
417209.1274060162
A R134a
B R23
P4 1530846.5342272525
0.8739737932766035
417209.1274060162
A R717
B R23
P4 1530846.5342272525
0.8564663946841126
417209.1274060162
A R290
B R23
P4 1530846.5342272525
0.8758034929157003
417209.1274060162
A R600
B R23
P4 1530846.5342272525
0.8669435641171691
492975.76701854524
A R134a
B R170
P4 1543865.1621415792
0.8806029376182987
492975.76701854524
A R717
B R170
P4 1543865.1621415792
0.8629015832258888
492975.76701854524
A R290
B R170
P4 1543865.1621415792
0.8824530524214212
492975.76701854524
A R600
B R170
P4 1543865.1621415792
0.8601045938418034
567157.2364834511
A R134a
B NitrousOxide
P4 1963060.1245940677
0.8736056180036209
567157.2364834511
A R717
B NitrousOxide
P4 1963060.1245940677
0.8561091706600834
567157.2364834511
A R290
B NitrousOxide
P4 1963060.1245940677
0.8754341646906707
567157.2364834511
A R600
B NitrousOxide
P4 1963060.1245940677
